In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
from rdkit import Chem
from cbiprep.pdbatoms import PDBAtoms
from cbiprep.matrix import GetTopologicalMatrix
from cbiprep.ligand_expo import LigandExpo
from cbiprep.atomtyper import AtomTyper, HybAtomTyper, GetAtomVector
import subprocess as sp

np.set_printoptions(edgeitems=10)

In [ ]:
pdb_atoms = PDBAtoms('pdb/4jn2.pdb.gz')

In [ ]:
pdb_atoms.get_ligand_names()

In [ ]:
ligand_atoms = pdb_atoms.get_ligand('4CC')

In [ ]:
len(ligand_atoms)

In [ ]:
protein_atoms = pdb_atoms.get_relevant_protein(ligand_atoms)

In [ ]:
len(protein_atoms)

In [ ]:
pocket_atoms = protein_atoms.get_pocket(ligand_atoms, thres=5)
open('pocket.pdb', 'wt').write(str(pocket_atoms)+'\n')

In [ ]:
len(pocket_atoms)

### Test distance-based adjacency matrix

### Padding

<img src="padding.png" width="40%" align="left" />

### GetTopologicalMatrix as a function

In [ ]:
mat = GetTopologicalMatrix(ligand_atoms, pocket_atoms)
mat

### Test atom typings

In [ ]:
expo_dic = LigandExpo()
ligand_mol = expo_dic.assign(ligand_atoms, '4CC')
print(Chem.MolToSmiles(ligand_mol))

In [ ]:
ligand_thres = 50
pocket_thres = 400

In [ ]:
atomtyper = HybAtomTyper

In [ ]:
types_vec = GetAtomVector(ligand_mol, pocket_atoms, atomtyper=atomtyper)

### Types Vec to One-Hot expression

In [ ]:
onehot = np.zeros((atomtyper.MAX, ligand_thres + pocket_thres), dtype=int)
for i, v in enumerate(types_vec):
    if 0 < v:
        onehot[v-1, i] = 1
onehot

### Supplement, for any extension

In [ ]:
for atom in ligand_mol.GetAtoms():
    idx = atom.GetIdx()
    an = atom.GetAtomicNum()
    el = atom.GetSymbol()
    arom = int(atom.GetIsAromatic())
    hyb = atom.GetHybridization()
    nH = atom.GetTotalNumHs()
    nH = f'({nH}H)' if nH != 0 else ''
    ch = atom.GetFormalCharge()
    ch = '(+)' if 0 < ch else '(-)' if ch < 0 else ''
    val = atom.GetTotalValence()
    deg = atom.GetTotalDegree()
    atomf = f'{idx:3d} {el}{ch}{nH}'
    typ = {Chem.HybridizationType.SP:1, Chem.HybridizationType.SP2:2, Chem.HybridizationType.SP3:3}.get(hyb)
    if arom:
        typ = 5
    print(f'{atomf:<14} hyb={typ} v,d={val},{deg}')

### Run Flare